In [1]:
import logging
import pickle
import numpy as np

In [2]:
#from instance_unchecked import Instance, InfeasibleOnBuilding
from instance import Instance, InfeasibleOnBuilding
from model import ModelSolver
#from column_generation_master_update import ColumnGeneration
from column_generation_heuristic_pricing import ColumnGeneration

In [3]:
def setup_logger(name, log_file, level):

    formatter = logging.Formatter('%(levelname)s %(message)s')
    handler = logging.FileHandler(log_file, mode='w')        
    handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.addHandler(handler)

    return logger

In [4]:
with open('instances.pickle', 'rb') as f:
    instances = pickle.load(f)

In [5]:
for i in instances:
    print(i)

('DATASET/networks/network_030_a.dat', 'DATASET/networks/network_030_a_rp_0.dat', 'DATASET/apps_merged/id_0000.dat', 'DATASET/apps_merged/id_0000_rp.dat', ['DATASET/apps/graph_06_d.dat', 'DATASET/apps/path_04_a.dat'], ['DATASET/apps/graph_06_d_rp_1.dat', 'DATASET/apps/path_04_a_rp_4.dat'])
('DATASET/networks/network_030_a.dat', 'DATASET/networks/network_030_a_rp_4.dat', 'DATASET/apps_merged/id_0001.dat', 'DATASET/apps_merged/id_0001_rp.dat', ['DATASET/apps/tree_03_b.dat', 'DATASET/apps/tree_11_a.dat'], ['DATASET/apps/tree_03_b_rp_0.dat', 'DATASET/apps/tree_11_a_rp_4.dat'])
('DATASET/networks/network_030_a.dat', 'DATASET/networks/network_030_a_rp_4.dat', 'DATASET/apps_merged/id_0002.dat', 'DATASET/apps_merged/id_0002_rp.dat', ['DATASET/apps/path_13_a.dat', 'DATASET/apps/tree_10_a.dat'], ['DATASET/apps/path_13_a_rp_4.dat', 'DATASET/apps/tree_10_a_rp_4.dat'])
('DATASET/networks/network_030_c.dat', 'DATASET/networks/network_030_c_rp_0.dat', 'DATASET/apps_merged/id_0003.dat', 'DATASET/apps_

In [6]:
"""
for idx,i in enumerate(instances):

    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i
    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}")
    
    I, A, c, P, a2 = Instance.build_network_structure_from_file(network_filename)

    min_path_len = min([len(P[i][j]) for i in range(I) for j in range(I)])
    max_path_len = max([len(P[i][j]) for i in range(I) for j in range(I)])
    avg_path_len = np.mean([len(P[i][j]) for i in range(I) for j in range(I)])
    std_path_len = np.std([len(P[i][j]) for i in range(I) for j in range(I)])
    print(min_path_len, avg_path_len, max_path_len, std_path_len)
"""

'\nfor idx,i in enumerate(instances):\n\n    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i\n    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}")\n    \n    I, A, c, P, a2 = Instance.build_network_structure_from_file(network_filename)\n\n    min_path_len = min([len(P[i][j]) for i in range(I) for j in range(I)])\n    max_path_len = max([len(P[i][j]) for i in range(I) for j in range(I)])\n    avg_path_len = np.mean([len(P[i][j]) for i in range(I) for j in range(I)])\n    std_path_len = np.std([len(P[i][j]) for i in range(I) for j in range(I)])\n    print(min_path_len, avg_path_len, max_path_len, std_path_len)\n'

In [ ]:
for idx,i in enumerate(instances):
    
    # da 10 app in poi size 40
    # if idx < 126:
    #    continue

    # da 10 app in poi size 50
    #if idx < 108 or idx > 125:
    #    continue

    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i
    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}\n")
    
    try:
        instance = Instance.build(
            network_filename, 
            network_rp_filename, 
            app_merged, 
            app_merged_rp
        )
    except InfeasibleOnBuilding:
        print("INFEASIBLE on building")
        continue

    
    print("\nCG :\n")
    logger = setup_logger(f"logger_{idx:03d}", f"logs/id_{idx:04d}.log", logging.DEBUG)
    
    try:
        cg = ColumnGeneration(
            N= len(app_filenames),
            filenames= (
                network_filename, network_rp_filename, 
                app_filenames, app_rp_filenames
            ),
            logger= logger
        )
        result = cg.execute()
    except Exception as e:
        print(f"ERROR in ColumnGeneration : {e}")
        raise RuntimeError()
    
    
    print()
    print(f"dual {result[0]} primal {result[1]}")

    # CG_DUAL_BOUND[idx] = result[0]
    # CG_PRIMAL_BOUND[idx] = result[1]
    # CG_TIME[idx] = result[2]
    
    print("\n" + "-" * 80 + "\n")

2 apps, DATASET/networks/network_030_a_rp_0.dat DATASET/apps_merged/id_0000.dat


CG :

app 0 
μ = 0.1 μ = 0.2 generated 100 columns
runs_count 342 , total time 0.008191787000896511 , avg time x run 2.395259356987284e-05
μ = 0.4 generated 54 columns
runs_count 500 , total time 0.15558366399659462 , avg time x run 0.00031116732799318925
μ = 0.6 generated 32 columns
runs_count 500 , total time 0.2772299299986116 , avg time x run 0.0005544598599972233
μ = 0.8 generated 32 columns
runs_count 500 , total time 0.2957653270002538 , avg time x run 0.0005915306540005077
μ = 1.0 generated 35 columns
runs_count 500 , total time 0.3778267459986182 , avg time x run 0.0007556534919972364
generated 253 feasible columns - time 1.12 s, actually added 144/253 , min_cost = 129 , avg_cost = 252.7193675889328

app 1 
μ = 0.1 generated 100 columns
runs_count 100 , total time 0.0023315459993682452 , avg time x run 2.3315459993682453e-05
μ = 0.2 generated 100 columns
runs_count 100 , total time 0.002372585998

In [8]:
"""
RESULTS_ROOT_NODE_RELAXATION = {}

for idx,i in enumerate(instances):

    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i
    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}\n")
    
    try:
        instance = Instance.build(
            network_filename, 
            network_rp_filename, 
            app_merged, 
            app_merged_rp
        )
    except InfeasibleOnBuilding:
        print("INFEASIBLE on building")
        continue

    RESULTS_ROOT_NODE_RELAXATION[idx] = ModelSolver.optimize_model_at_root_node(instance)
"""

'\nRESULTS_ROOT_NODE_RELAXATION = {}\n\nfor idx,i in enumerate(instances):\n\n    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i\n    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}\n")\n    \n    try:\n        instance = Instance.build(\n            network_filename, \n            network_rp_filename, \n            app_merged, \n            app_merged_rp\n        )\n    except InfeasibleOnBuilding:\n        print("INFEASIBLE on building")\n        continue\n\n    RESULTS_ROOT_NODE_RELAXATION[idx] = ModelSolver.optimize_model_at_root_node(instance)\n'

In [9]:
"""
RESULTS_ROOT_NODE_RELAXATION = {}
RESULTS_COMPACT_MODEL = {}
CG_DUAL_BOUND = {}
CG_PRIMAL_BOUND = {}
CG_TIME = {}

for idx,i in enumerate(instances):

    if idx <= 293:
        continue

    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i
    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}\n")
    
    try:
        instance = Instance.build(
            network_filename, 
            network_rp_filename, 
            app_merged, 
            app_merged_rp
        )
    except InfeasibleOnBuilding:
        print("INFEASIBLE on building")
        RESULTS_ROOT_NODE_RELAXATION[idx] = "INFEASIBLE on building"
        continue
    
    print("\nROOT NODE RELAXATION :\n")
    RESULTS_ROOT_NODE_RELAXATION[idx] = ModelSolver.optimize_model_at_root_node(instance)
    print()

    print("\nCOMPACT MODEL :\n")
    RESULTS_COMPACT_MODEL[idx] = ModelSolver.optimize_model(instance)

    
    # print("\nCG :\n")
    # logger = setup_logger(f"logger_{idx:03d}", f"logs/id_{idx:04d}.log", logging.DEBUG)
    
    # try:
    #     cg = ColumnGeneration(
    #         N= len(app_filenames),
    #         filenames= (
    #             network_filename, network_rp_filename, 
    #             app_filenames, app_rp_filenames
    #         ),
    #         logger= logger
    #     )
    #     result = cg.execute()
    # except Exception as e:
    #     print(f"ERROR in ColumnGeneration : {e}")
    #     raise RuntimeError()
    
    
    # print()
    # print(f"dual {result[0]} primal {result[1]}")

    # CG_DUAL_BOUND[idx] = result[0]
    # CG_PRIMAL_BOUND[idx] = result[1]
    # CG_TIME[idx] = result[2]
    

    print("\n" + "-" * 80 + "\n")


# with open('cg_results.pickle', 'wb') as f:
#     pickle.dump(tuple([RESULTS_ROOT_NODE_RELAXATION, CG_DUAL_BOUND, CG_PRIMAL_BOUND, CG_TIME]), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('compact_model_100_120_results.pickle', 'wb') as f:
    pickle.dump(RESULTS_COMPACT_MODEL, f, protocol=pickle.HIGHEST_PROTOCOL)
"""

'\nRESULTS_ROOT_NODE_RELAXATION = {}\nRESULTS_COMPACT_MODEL = {}\nCG_DUAL_BOUND = {}\nCG_PRIMAL_BOUND = {}\nCG_TIME = {}\n\nfor idx,i in enumerate(instances):\n\n    if idx <= 293:\n        continue\n\n    network_filename, network_rp_filename, app_merged, app_merged_rp, app_filenames, app_rp_filenames = i\n    print(f"{len(app_filenames)} apps, {network_rp_filename} {app_merged}\n")\n    \n    try:\n        instance = Instance.build(\n            network_filename, \n            network_rp_filename, \n            app_merged, \n            app_merged_rp\n        )\n    except InfeasibleOnBuilding:\n        print("INFEASIBLE on building")\n        RESULTS_ROOT_NODE_RELAXATION[idx] = "INFEASIBLE on building"\n        continue\n    \n    print("\nROOT NODE RELAXATION :\n")\n    RESULTS_ROOT_NODE_RELAXATION[idx] = ModelSolver.optimize_model_at_root_node(instance)\n    print()\n\n    print("\nCOMPACT MODEL :\n")\n    RESULTS_COMPACT_MODEL[idx] = ModelSolver.optimize_model(instance)\n\n    \n

In [10]:
#RESULTS_COMPACT_MODEL

In [11]:
# with open('cg_results.pickle', 'rb') as f:
#     t = pickle.load(f)
# RESULTS_ROOT_NODE_RELAXATION, CG_DUAL_BOUND, CG_PRIMAL_BOUND, CG_TIME = t

In [12]:
# RESULTS_ROOT_NODE_RELAXATION, CG_DUAL_BOUND, CG_PRIMAL_BOUND, CG_TIME

In [13]:
# with open('root_node_relaxations_30_120_results.pickle', 'wb') as f:
#     pickle.dump(RESULTS_ROOT_NODE_RELAXATION, f, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
"""
# count = 0
# count_drmp_success = 0

for idx,i in enumerate(instances):
    
    print(f"{i[0][17:-4]}\t{len(i[4])}\t", end=' ')
    if len(RESULTS_ROOT_NODE_RELAXATION[idx]) == 2:
        print(f"INFEASIBLE\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}")
    else:
        #print(RESULTS_ROOT_NODE_RELAXATION[idx])
        assert len(RESULTS_ROOT_NODE_RELAXATION[idx]) == 3
        print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][2]:.2f}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}")


# print()
# print(f"{count_drmp_success} / {count} DRMP successes")
"""

'\n# count = 0\n# count_drmp_success = 0\n\nfor idx,i in enumerate(instances):\n    \n    print(f"{i[0][17:-4]}\t{len(i[4])}\t", end=\' \')\n    if len(RESULTS_ROOT_NODE_RELAXATION[idx]) == 2:\n        print(f"INFEASIBLE\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}")\n    else:\n        #print(RESULTS_ROOT_NODE_RELAXATION[idx])\n        assert len(RESULTS_ROOT_NODE_RELAXATION[idx]) == 3\n        print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][2]:.2f}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}")\n\n\n# print()\n# print(f"{count_drmp_success} / {count} DRMP successes")\n'

In [15]:
"""
with open('compact_model_100_120_results.pickle', 'rb') as f:
    RESULTS_COMPACT_MODEL = pickle.load(f)

for k,v in RESULTS_COMPACT_MODEL.items():
    print(f"{instances[k][0][17:-4]}\t{len(instances[k][4])}\t", end=' ')
    #print(v)
    if len(v) == 3:
        if v[1] == "INFEASIBLE":
            print(f"INFEASIBLE\t{v[0]:.2f}")
        else:
            print("TIME_LIMIT_>10min")
    else:
        print(f"{v[0]:.2f}\t{v[3]}")

        # count += 1
        # if CG_PRIMAL_BOUND[idx] is not None:
        #     count_drmp_success += 1

        # if CG_DUAL_BOUND[idx] == "INFEASIBLE INSTANCE":
        #     print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][3]}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}\tINFEASIBLE\t{CG_TIME[idx]:.2f}\t{CG_PRIMAL_BOUND[idx]}")
        # else:
        #     print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][3]}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}\t{CG_DUAL_BOUND[idx]:.2f}\t{CG_TIME[idx]:.2f}\t{CG_PRIMAL_BOUND[idx]}")  
"""


'\nwith open(\'compact_model_100_120_results.pickle\', \'rb\') as f:\n    RESULTS_COMPACT_MODEL = pickle.load(f)\n\nfor k,v in RESULTS_COMPACT_MODEL.items():\n    print(f"{instances[k][0][17:-4]}\t{len(instances[k][4])}\t", end=\' \')\n    #print(v)\n    if len(v) == 3:\n        if v[1] == "INFEASIBLE":\n            print(f"INFEASIBLE\t{v[0]:.2f}")\n        else:\n            print("TIME_LIMIT_>10min")\n    else:\n        print(f"{v[0]:.2f}\t{v[3]}")\n\n        # count += 1\n        # if CG_PRIMAL_BOUND[idx] is not None:\n        #     count_drmp_success += 1\n\n        # if CG_DUAL_BOUND[idx] == "INFEASIBLE INSTANCE":\n        #     print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][3]}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}\tINFEASIBLE\t{CG_TIME[idx]:.2f}\t{CG_PRIMAL_BOUND[idx]}")\n        # else:\n        #     print(f"{RESULTS_ROOT_NODE_RELAXATION[idx][3]}\t{RESULTS_ROOT_NODE_RELAXATION[idx][0]:.2f}\t{CG_DUAL_BOUND[idx]:.2f}\t{CG_TIME[idx]:.2f}\t{CG_PRIMAL_BOUND[idx]}")  \n'